# Diccionario Quechua

## Planificación

1. [Definición del problema]()
1. [Plantear los objetivos]()
1. [Obtener los datos]()
1. [Preparar y optimizar los datos]()
1. [Procesar y analizar los datos]()
1. [Conclusiones]()

### Definición del problema

El idioma quechua tiene dos vertientes en el uso de tres o cinco vocales, 
así como el número de sufijos usados.

### Plantear los objetivos

1. Describir las palabras en idioma Quechua según vocales y sufijos.
2. Listar las palabras en el idioma Quechua.
3. Clasificar según las tres vocales diferenciando los sufijos.

### Obtener los datos

El idioma Quechua cuenta con presencia en Internet y se dispone de diccionarios en varios formatos:

* PDF - Publicaciones de instituciones o autores individuales
* Datos - En formato Texto o Excel, para su fácil uso en Internet

### Preparar y optimizar los datos

### Procesar y analizar los datos

### Conclusiones